In [31]:
pip install requests beautifulsoup4


Note: you may need to restart the kernel to use updated packages.


In [47]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [74]:
import requests
import json
import time
import datetime

event_id = "1074826"
race_id = "2449286"
base_results_url = f"https://reignite-api.athlinks.com/event/{event_id}/race/{race_id}/results"
base_runner_url = f"https://reignite-api.athlinks.com/event/{event_id}/race/{race_id}/bib/{{bib_number}}/result"
all_runners_data = []
from_param = 0
limit = 50

print("Starting to fetch runner data with split times...")

while True:
    params = {"from": from_param, "limit": limit}
    print(f"Fetching results from index: {from_param}")
    try:
        response = requests.get(base_results_url, params=params)
        response.raise_for_status()
        try:
            data = response.json()
            if "intervals" in data and isinstance(data["intervals"], list) and len(data["intervals"]) > 0 and "results" in data["intervals"][0] and isinstance(data["intervals"][0]["results"], list):
                runners = data["intervals"][0]["results"]
                print(f"Fetched {len(runners)} runners on this page.")

                for runner in runners:
                    runner_info = {
                        "name": runner.get("displayName"),
                        "bib": runner.get("bib"),
                        "gender": runner.get("gender"),
                        "overall_time_ms": runner.get("gunTimeInMillis"), # Keep milliseconds for now
                        "overall_time_readable": str(datetime.timedelta(milliseconds=runner.get("gunTimeInMillis"))),
                        "splits": {}
                    }

                    bib_number = runner.get("bib")
                    if bib_number:
                        runner_detail_url = base_runner_url.format(bib_number=bib_number)
                        try:
                            detail_response = requests.get(runner_detail_url)
                            detail_response.raise_for_status()
                            detail_data = detail_response.json()
                            if "intervals" in detail_data and isinstance(detail_data["intervals"], list):
                                for interval in detail_data["intervals"]:
                                    interval_name = interval.get("name")
                                    chip_time_millis = interval.get("chipTimeInMillis")
                                    if interval_name and chip_time_millis is not None:
                                        time_obj = datetime.timedelta(milliseconds=chip_time_millis)
                                        readable_time = str(time_obj)
                                        runner_info["splits"][interval_name] = readable_time

                        except requests.exceptions.RequestException as e:
                            print(f"Error fetching details for bib {bib_number}: {e}")
                        except json.JSONDecodeError:
                            print(f"Error decoding JSON for bib {bib_number}")

                    all_runners_data.append(runner_info)

                from_param += limit
                time.sleep(1)

            else:
                print("Unexpected JSON structure for runner results.")
                break

        except json.JSONDecodeError:
            print("Error decoding JSON response:")
            print(response.text)
            break

    except requests.exceptions.RequestException as e:
        print(f"Error fetching page {from_param // limit + 1}: {e}")
        break

print(f"\nSuccessfully collected data for {len(all_runners_data)} runners.")
# You can now process the 'all_runners_data' list.
# For example, print the first runner's data with splits:
if all_runners_data:
    print("\nExample of the first runner's data:")
    print(json.dumps(all_runners_data[0], indent=4))

Starting to fetch runner data with split times...
Fetching results from index: 0
Fetched 50 runners on this page.
Fetching results from index: 50
Fetched 50 runners on this page.
Fetching results from index: 100
Fetched 50 runners on this page.
Fetching results from index: 150
Fetched 50 runners on this page.
Fetching results from index: 200
Fetched 50 runners on this page.
Fetching results from index: 250
Fetched 4 runners on this page.
Fetching results from index: 300
Unexpected JSON structure for runner results.

Successfully collected data for 254 runners.

Example of the first runner's data:
{
    "name": "Calvin Lehn",
    "bib": "263",
    "gender": "M",
    "overall_time_ms": 9522000,
    "overall_time_readable": "2:38:42",
    "splits": {
        "Full Course": "2:38:41",
        "13.1 Miles": "1:17:46"
    }
}


In [76]:
import pandas as pd

#'all_runners_data' list is still populated

csv_file_path = 'race_results2.csv'

if 'all_runners_data' in locals() and all_runners_data:
    df = pd.DataFrame(all_runners_data)

    # Separate the 'splits' dictionary into individual columns
    splits_df = pd.json_normalize(df['splits'])

    # Concatenate the main DataFrame with the splits DataFrame
    df = pd.concat([df.drop('splits', axis=1), splits_df], axis=1)

    # Save the DataFrame to a CSV file
    df.to_csv(csv_file_path, index=False, encoding='utf-8')

    print(f"Data successfully written to {csv_file_path} using pandas (without re-fetching).")
else:
    print("Error: The 'all_runners_data' list does not exist or is empty. Please run the data fetching part first.")

Data successfully written to race_results2.csv using pandas (without re-fetching).
